In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
#Import Data in XLSX format
data = pd.read_excel('Training Data.xlsx')
#Cleaning Transfer Value Columns
def cleaning_transfer_value_columns(data):
    #Remove Rows that has 'Not For Sale' as the Transfer Value
    mask = data['Transfer Value'] != 'Not for Sale'
    data = data[mask].copy()
    #Split Transfer Value Columns
    data[['Lower Transfer Value','Higher Transfer Value']] = data['Transfer Value'].str.split(' - ',expand=True)
    #Cleaning Lower Transfer Value and Higher Transfer Value
    data['Lower Transfer Value'] = data['Lower Transfer Value'].str.replace('$', '', regex=False).str.replace('\xa0', '', regex=False).str.replace(' ', '', regex=False).str.replace('.', '', regex=False)
    data['Higher Transfer Value'] = data['Higher Transfer Value'].str.replace('$', '', regex=False).str.replace('\xa0', '', regex=False).str.replace(' ', '', regex=False).str.replace('.', '', regex=False)
    #Putting the Correct Number of 0 
    data['Lower Transfer Value'] = data['Lower Transfer Value'].str.replace('K', '000', regex=False).str.replace('M', '000000', regex=False)
    data['Higher Transfer Value'] = data['Higher Transfer Value'].str.replace('K', '000', regex=False).str.replace('M', '000000', regex=False)
    #Filling None Value
    data['Higher Transfer Value'] = data['Higher Transfer Value'].fillna(data['Lower Transfer Value'])
    data['Lower Transfer Value'] = data['Lower Transfer Value'].fillna(data['Higher Transfer Value'])
    #Changing Data Type From Object to FLoat
    data[['Lower Transfer Value', 'Higher Transfer Value']] = data[['Lower Transfer Value', 'Higher Transfer Value']].apply(pd.to_numeric)
    #Making a New Column for Average Transfer Value
    data['Average Transfer Value'] = (data['Lower Transfer Value'] + data['Higher Transfer Value']) / 2
    return data
data = cleaning_transfer_value_columns(data)
#Korelasi
print(data[['CA','PA','Age']].corr())

           CA        PA       Age
CA   1.000000  0.875231  0.220476
PA   0.875231  1.000000  0.075409
Age  0.220476  0.075409  1.000000


$$
\text{FIRST MODEL}
$$
$$
\text{Transfer Value} = e^{\text{intercept}} \cdot \text{Age}^{\beta_1} \cdot \text{CA}^{\beta_2} \cdot \text{PA}^{\beta_3}
$$

In [3]:
#Filtering so Player with 'Bad' Potential is not in Calculation
data_training_1 = data.copy()
mask = (data_training_1['PA'] >= 100) & (data_training_1['Lower Transfer Value'] > 0)
data_training_1 = data_training_1[mask].copy()
#Defining Input 
input_1 = np.log(data_training_1[['Age','CA','PA']])
output_1 = np.log(data_training_1['Lower Transfer Value'])
#Splitting the Data
input_train_1, input_test_1, output_train_1, output_test_1 = train_test_split(
    input_1, output_1, test_size=0.2, random_state=42)
#Picking the Model and Fitting the Data
model_1 = LinearRegression()
model_1.fit(input_train_1, output_train_1)
#Model Coefficient
intercept_ = np.exp(model_1.intercept_)
print("intercept (b0) :", intercept_)
print("Coefficients (b1)", model_1.coef_[0])
print("Coefficients (b2)", model_1.coef_[1])
print("Coefficients (b3)", model_1.coef_[2])
print()
#Transfer Value Prediction
prediction_test_1 = model_1.predict(input_test_1)
#R Squared Value
mse_1 = mean_squared_error(output_test_1,prediction_test_1)
r_squared_1= r2_score(output_test_1,prediction_test_1)
print("Lower Transfer Value Model")
print('MSE :',mse_1)
print('R Squared :',r_squared_1)

intercept (b0) : 1.5062264825084219e-21
Coefficients (b1) -5.5799432904661685
Coefficients (b2) 10.68887630822278
Coefficients (b3) 5.880307069493838

Lower Transfer Value Model
MSE : 2.8300611758255343
R Squared : 0.7501596003469526


In [4]:
#Making Clean Data Frame
clean_training_data_1 = data_training_1[['Name','Age','CA','PA','Lower Transfer Value']].copy()
full_input_1 = input_1
full_prediction_1 = model_1.predict(full_input_1)
#Importing Prediction Value
clean_training_data_1['Prediction Lower Transfer Value'] = np.exp(full_prediction_1)
#Remove the 1 Dollar that is Added
data_training_1['Lower Transfer Value'] = data_training_1['Lower Transfer Value'] - 1
#Making the Ratio
clean_training_data_1['Lower Ratio'] = clean_training_data_1['Lower Transfer Value'] / clean_training_data_1['Prediction Lower Transfer Value']
#Columns Cleaning
new_column_order = ['Name', 'Age', 'CA','PA','Lower Ratio','Lower Transfer Value','Prediction Lower Transfer Value']
clean_training_data_1 = clean_training_data_1[new_column_order]
#Import to Excel
clean_training_data_1.to_excel('_TrainingData_Transfe_Value_Model1.xlsx', index=False, sheet_name='sheet')

#### **IMPORT SCOUT DATA**

In [5]:
#Importing Scout Data
scout_data_1 = pd.read_excel('SCOUT DATA.xlsx')
#Cleaning Transfer Value Columns
scout_data_1 = cleaning_transfer_value_columns(scout_data_1)

In [6]:
#Creating New Data Frame
clean_scout_data_1 = scout_data_1[['Name','Age','CA','PA','Lower Transfer Value']].copy()
#Inputs to Predict The Model
scout_data_input_1 = np.log(clean_scout_data_1[['Age','CA','PA']])
#Predicting Transfer Value using Scout Data
scout_data_prediction_1 = model_1.predict(scout_data_input_1)
#Importing Prediction Value
clean_scout_data_1['Prediction Lower Transfer Value'] = np.exp(scout_data_prediction_1)
#Making the Ratio
clean_scout_data_1['Lower Ratio'] = clean_scout_data_1['Lower Transfer Value'] / clean_scout_data_1['Prediction Lower Transfer Value']
#Columns Cleaning
new_column_order = ['Name', 'Age', 'CA','PA','Lower Ratio','Lower Transfer Value','Prediction Lower Transfer Value']
clean_scout_data_1 = clean_scout_data_1[new_column_order]
#Import to Excel
clean_scout_data_1.to_excel('_Full_ScoutData_TransferValue_Model1.xlsx', index=False, sheet_name='sheet')

In [7]:
#Filter Data
mask = clean_scout_data_1['PA']>=155
clean_scout_data_1 = clean_scout_data_1[mask].copy()
mask = clean_scout_data_1['Age']<=20
clean_scout_data_1 = clean_scout_data_1[mask].copy()
mask = clean_scout_data_1['Lower Ratio']<=2
clean_scout_data_1 = clean_scout_data_1[mask].copy()

#Sorted by Ratio
clean_scout_data_1 = clean_scout_data_1.sort_values(by='Lower Ratio')
#Import to Excel
clean_scout_data_1.to_excel('_Filtered_ScoutData_TransferValue_Model1.xlsx', index=False, sheet_name='sheet')

$$
\text{FIRST MODEL}
$$
$$
\text{RANDOM FOREST REGRESSION}
$$

In [8]:
#Filtering so Player with 'Bad' Potential is not in Calculation
data_training_2 = data.copy()
mask = (data_training_2['PA'] >= 100) & (data_training_2['Lower Transfer Value'] > 0)
data_training_2 = data_training_2[mask].copy()
#Defining Input 
input_2 = data_training_2[['Age','CA','PA']]
output_2 = data_training_2['Lower Transfer Value']
#Splitting the Data
input_train_2, input_test_2, output_train_2, output_test_2 = train_test_split(
    input_2, output_2, test_size=0.2, random_state=42)
#Picking the Model and Fitting the Data
model_2 = RandomForestRegressor(n_estimators=200, random_state=42)
model_2.fit(input_train_2, output_train_2)
#Transfer Value Prediction
prediction_test_2 = model_2.predict(input_test_2)
#Evaluate the Model
mse_2 = mean_squared_error(output_test_2, prediction_test_2)
r_squared_2 = r2_score(output_test_2, prediction_test_2)
print("Lower Transfer Value Model")
print('MSE :',mse_2)
print('R Squared :',r_squared_2)

Lower Transfer Value Model
MSE : 768356426645981.8
R Squared : 0.46570632482263563


In [9]:
#Creating New Data Frame
clean_scout_data_2 = scout_data_1[['Name','Age','CA','PA','Lower Transfer Value']].copy()
#Inputs to Predict The Model
scout_data_input_2 = clean_scout_data_2[['Age','CA','PA']]
#Predicting Transfer Value using Scout Data
scout_data_prediction_2 = model_2.predict(scout_data_input_2)
#Importing Prediction Value
clean_scout_data_2['Prediction Lower Transfer Value'] = scout_data_prediction_2
#Making the Ratio
clean_scout_data_2['Lower Ratio'] = clean_scout_data_2['Lower Transfer Value'] / clean_scout_data_2['Prediction Lower Transfer Value']
#Columns Cleaning
new_column_order = ['Name', 'Age', 'CA','PA','Lower Ratio','Lower Transfer Value','Prediction Lower Transfer Value']
clean_scout_data_2 = clean_scout_data_2[new_column_order]
#Import to Excel
clean_scout_data_2.to_excel('_Full_ScoutData_TransferValue_Model2.xlsx', index=False, sheet_name='sheet')

In [10]:
#Filter Data
mask = clean_scout_data_2['PA']>=160
clean_scout_data_2 = clean_scout_data_2[mask].copy()
mask = clean_scout_data_2['Age']<=20
clean_scout_data_2 = clean_scout_data_2[mask].copy()
mask = clean_scout_data_2['Lower Transfer Value']<=15000000
clean_scout_data_2 = clean_scout_data_2[mask].copy()

#Sorted by Ratio
clean_scout_data_2 = clean_scout_data_2.sort_values(by='Lower Ratio')
#Import to Excel
clean_scout_data_2.to_excel('_Filtered_ScoutData_TransferValue_Model2.xlsx', index=False, sheet_name='sheet')